# 문서 검색 챗봇 만들기 LangChain

In [5]:
import openai
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS
import os

In [6]:
!wget https://github.com/kairess/toy-datasets/raw/master/Demian.pdf

'wget'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [7]:
import requests

url = "https://github.com/kairess/toy-datasets/raw/master/Demian.pdf"
response = requests.get(url)

with open("Demian.pdf", "wb") as f:
    f.write(response.content)

print("다운로드 완료")


다운로드 완료


# [주의] 레거시 코드 (실행하지 마세요)

## Preprocess a PDF file

In [8]:
reader = PdfReader("Demian.pdf")

raw_text = ""

for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

raw_text = raw_text[:1000] # 안전장치

## Summarize 요약

In [9]:
from langchain import OpenAI
from langchain.chains import AnalyzeDocumentChain
from langchain.chains.summarize import load_summarize_chain

llm = OpenAI(temperature=0)
summary_chain = load_summarize_chain(llm, chain_type="map_reduce")

summarize_document_chain = AnalyzeDocumentChain(combine_docs_chain=summary_chain)

ValidationError: 1 validation error for OpenAI
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'temperature': 0, 'model...ne, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [ ]:
summarize_document_chain.run(raw_text)

" Hermann Hesse's Demian follows the journey of a young man as he discovers his true identity and learns to accept himself. He meets Max Demian, who helps him to understand the importance of self-reliance and the power of his own ideas. Through Demian's influence, the protagonist learns to accept his place in the world and to embrace his own unique story. Along the way, he meets Pistorius, who teaches him about Abraxas, and Eva, Demian's mother, who encourages him to find his dream and remain faithful to it. When war is declared, Sinclair goes to the front and finds something akin to humanity beneath the surface of the war. In the end, he finds comfort in the knowledge that his own image now resembles his friend and leader."

### How much?

- text-davinci, 6 requests
- 101,611 prompt + 8,930 completion = 110,541 tokens
- $0.47 == ₩611

## Question Answering 질문 답변

1~3분 소요

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI

model = ChatOpenAI(model="gpt-4") # gpt-3.5-turbo, gpt-4, gpt-4o

qa_chain = load_qa_chain(model, chain_type="map_reduce")
qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)

In [ ]:
qa_document_chain.run(
    input_document=raw_text,
    question="싱클레어를 괴롭힌 사람은?")

'싱클레어를 괴롭힌 사람은 Franz Kromer입니다.'

In [ ]:
qa_document_chain.run(
    input_document=raw_text,
    question="싱클레어는 데미안을 어디서 어떻게 만났지?")

'제공된 텍스트에서는 싱클레어가 데미안을 만난 경위에 대한 구체적인 정보가 제공되지 않습니다. 따라서 이 질문에 대한 답변을 제공할 수 없습니다.'

In [ ]:
qa_document_chain.run(
    input_document=raw_text,
    question="데미안의 외모를 묘사해봐")

'저는 주어진 텍스트에서 데미안의 외모에 대한 구체적인 묘사를 찾을 수 없었습니다. 따라서 데미안의 외모에 대한 정보를 제공할 수 없습니다.'

### GPT-4

11분 소요

- gpt-4-0314, 28 requests
    - 29,503 prompt + 1,111 completion = 30,614 tokens tokens
- gpt-4-0314, 42 requests
    - 44,041 prompt + 1,784 completion = 45,825 tokens
- gpt-4-0314, 10 requests
    - 10,652 prompt + 865 completion = 11,517 tokens


87,956 tokens total == $5.28

> $0.06 / 1K tokens

In [ ]:
model = ChatOpenAI(model="gpt-4") # gpt-3.5-turbo, gpt-4, gpt-4o

qa_chain = load_qa_chain(model, chain_type="map_reduce")
qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)

qa_document_chain.run(
    input_document=raw_text,
    question="데미안의 외모를 묘사해봐")

'데미안의 외모는 다음과 같이 묘사되어 있습니다: 근접한 갈색 머리카락, 반여성적인 입, 이상한 밝기를 가진 강한 이마, 그리고 넓게 벌어진 녹색 눈동자로, 오른쪽 눈이 왼쪽 눈보다 약간 높게 위치해 있습니다. 또한, 데미안은 군복을 입고 은색 회색 망토를 입은 모습이 기묘하고 이상한 것처럼 묘사되어 있습니다.'

# 개선 코드 (여기서부터 실행하세요)

In [ ]:
!pip install -q langchain langchain-community openai pypdf chromadb tiktoken langchain-openai

In [9]:
import os
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.chains.summarize import load_summarize_chain

## OpenAI API Key

https://platform.openai.com/account/api-keys

In [ ]:
os.environ["OPENAI_API_KEY"] = "{YOUR_KEY}"

## 문서 다운로드 및 로드

In [12]:
!wget https://github.com/kairess/toy-datasets/raw/master/Demian.pdf

--2025-06-04 03:24:07--  https://github.com/kairess/toy-datasets/raw/master/Demian.pdf
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kairess/toy-datasets/master/Demian.pdf [following]
--2025-06-04 03:24:08--  https://raw.githubusercontent.com/kairess/toy-datasets/master/Demian.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5683449 (5.4M) [application/octet-stream]
Saving to: ‘Demian.pdf.1’

Demian.pdf.1        100%[===================>]   5.42M  --.-KB/s    in 0.02s   

2025-06-04 03:24:08 (232 MB/s) - ‘Demian.pdf.1’ saved [5683449/5683449]



In [23]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Demian.pdf")
documents = loader.load()


print(f"전체 페이지 수: {len(documents)}")

full_text = "\n".join([doc.page_content for doc in documents[:10]]) # 10페이지 제한 (비용, 시간)

full_text[:1000]

전체 페이지 수: 182


"DEMIAN \n• \nDownloaded from https://www.holybooks.com\nHERMANN \nHESSE \n• DEMIAN \n* \nTranslated by W. J. Strachan \nLondon \nDownloaded from https://www.holybooks.com\nPrologue \nI cannot tell my story without going a long way back. \nIf it were possible I would go back much farther still to \nthe very earliest years of my childhood and beyond them \nto my family origins. \nWhen poets write novels they are apt to behave as if \nthey were gods, with the power to look beyond and com\xad\nprehend any human story and serve it up as if the \nAlmighty himself, omnipresent, were relating it in all \nits naked truth. That I am no more able to do than the \npoets. But my story is more important to me than any \npoet's story to him, for it is my own-and it is the story \nof a huffian being-not an invented, idealised person \nbut a real, live, uniq:-e being. What constitutes a real, \nlive human being is more of a mystery than ever these \ndays, and men-each one of whom is a valuable, unique

## 문서 분절

In [24]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)
texts = text_splitter.create_documents([full_text])

print("===== PAGE 2 =====")
print(texts[2])
print("===== PAGE 3 =====")
print(texts[3])

===== PAGE 2 =====
page_content='creature, deserving the \ltmOSt attention. In each indi­
vidual the spirit is made 'flesh, in each one the whole of 
creation suffers, in each one a Saviour is crucified. 
Few people nowadays know what man is. Many feel it 
intuitively and die more easily for that reason, just as I 
shall die more easily when I have completed this story. 
I cannot call myself a scholar. I have always been and 
still am a seeker but I no longer do my seeking among 
the stars or in books. I am beginning to hear the lessons 
which whisper in my blood. Mine is not a pleasant story, 
it does not possess the gentle harmony of invented tales; 
like the lives of all men who have given up trying to 
deceive themselves, it is a mixture of nonsense and 
chaos, madness and dreams. 
The life of every man is a way to himself, an attempt 
at a way, the suggestion of a path. No man has ever 
been utterly himself, yet every man strives to be so, the 
dull. the intelligent, each one as b

## 임베딩 및 벡터 데이터베이스 생성

In [33]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# OpenAI 임베딩 초기화
#embeddings = OpenAIEmbeddings()

# 벡터 데이터베이스 생성 및 유지
vectorstore = Chroma.from_documents(texts, embeddings, persist_directory="./chroma_db")
# vectorstore.persist()

## 1. 요약 체인 (Summarize Chain)

1. "Map" 단계

- 긴 문서를 작은 조각들로 나눕니다.
- 각 조각에 대해 개별적으로 요약을 생성합니다.


2. "Reduce" 단계

- "Map" 단계에서 생성된 모든 개별 요약들을 모읍니다.
- 이 요약들을 하나의 최종 요약으로 결합합니다.

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFaceHub

# Hugging Face 허브에서 무료 모델 사용
llm = HuggingFaceHub(
    repo_id="google/flan-t5-base",  # 또는 'facebook/bart-large-cnn'
    model_kwargs={"temperature": 0.7, "max_length": 512}
)

# 요약 체인 재설정
from langchain.chains.summarize import load_summarize_chain
summarize_chain = load_summarize_chain(llm, chain_type="map_reduce")
summary = summarize_chain.run(texts)


In [ ]:
# 언어 모델 초기화
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# 요약 체인 설정
summarize_chain = load_summarize_chain(llm, chain_type="map_reduce")

# 문서 요약 생성
summary = summarize_chain.run(texts)

# 요약 결과 출력
summary

## 2. 질문응답 체인 (QA Chain)

In [ ]:
# RetrievalQA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

question = "싱클레어를 괴롭힌 사람의 이름은?"

result = qa_chain.invoke({"query": question})

answer = result["result"]

answer

'싱클레어를 괴롭힌 사람의 이름은 프란츠 크로머(Franz Kromer)입니다.'

In [ ]:
sources = result["source_documents"]

# 출처 출력 (상위 3개 출처만 표시)
for idx, source in enumerate(sources[:3]):
    print(f"===== 출처 {idx + 1} =====")
    print(source)

===== 출처 1 =====
page_content='creature, deserving the \ltmOSt attention. In each indi­
vidual the spirit is made 'flesh, in each one the whole of 
creation suffers, in each one a Saviour is crucified. 
Few people nowadays know what man is. Many feel it 
intuitively and die more easily for that reason, just as I 
shall die more easily when I have completed this story. 
I cannot call myself a scholar. I have always been and 
still am a seeker but I no longer do my seeking among 
the stars or in books. I am beginning to hear the lessons 
which whisper in my blood. Mine is not a pleasant story, 
it does not possess the gentle harmony of invented tales; 
like the lives of all men who have given up trying to 
deceive themselves, it is a mixture of nonsense and 
chaos, madness and dreams. 
The life of every man is a way to himself, an attempt 
at a way, the suggestion of a path. No man has ever 
been utterly himself, yet every man strives to be so, the 
dull. the intelligent, each one as bes

In [ ]:
# RetrievalQA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

question = "데미안의 외모는?"

result = qa_chain.invoke({"query": question})

answer = result["result"]

answer

'죄송합니다만, 제공된 문맥에서는 데미안의 외모에 대한 구체적인 설명이 없습니다. 데미안의 외모에 대한 정보가 필요하다면, 원본 텍스트의 다른 부분을 참조하시거나 책을 직접 읽어보시는 것이 좋겠습니다.'

In [ ]:
sources = result["source_documents"]

# 출처 출력 (상위 3개 출처만 표시)
for idx, source in enumerate(sources[:3]):
    print(f"===== 출처 {idx + 1} =====")
    print(source)

===== 출처 1 =====
page_content='9 
Downloaded from https://www.holybooks.com
DEMIAN 
alone was the right, good, worthy thing; and yet I found 
the part of the story which was played among the wicked 
and lost souls far more-. alluring. "If it had been per­
missible to speak ou,J: and confess, I should have admitted 
that it often seemed a shame to me that the Prodigal 
Son should atone and be 'found' again-though this 
feeling was only vaguely present deep down within me 
like a presentiment or possibility. When I pictured the 
devil to myself, I found no difficulty in visualizing him 
in the street below, disguised or undisguised, or at the 
fair or in a tavern but never at home. 
My sisters belonged likewise to the world of light. It 
often seemed to me that they were closer in temperament 
to father and mother, better and more refined and with 
fewer faults than I. Of course they had their defects and 
their vagaries but these did not appear to me to go very 
deep. It was not as with

# [보너스] 엑셀, CSV 검색, Aggregation

In [ ]:
import pandas as pd

df = pd.read_csv("https://github.com/kairess/toy-datasets/raw/master/titanic.csv")

df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
!pip install -q langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 kB 3.6 MB/s eta 0:00:00


In [ ]:
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain.llms import OpenAI

llm_2 = OpenAI(temperature=0)

agent = create_pandas_dataframe_agent(llm_2, df, verbose=True, allow_dangerous_code=True)

agent.run("how many rows are there?")

<ipython-input-17-712acbc15fff>:4: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  llm_2 = OpenAI(temperature=0)




> Entering new AgentExecutor chain...
Thought: I need to count the number of rows in the dataframe
Action: python_repl_ast
Action Input: len(df)891891 is the number of rows in the dataframe
Final Answer: 891

> Finished chain.


'891'

In [ ]:
agent.run("행이 몇 개지?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of rows in the dataframe
Action: python_repl_ast
Action Input: len(df)891891 rows
Final Answer: 891 rows

> Finished chain.


'891 rows'

In [ ]:
agent.run("승객들의 평균 연령은?")



> Entering new AgentExecutor chain...
Thought: I need to find the average age of the passengers.
Action: python_repl_ast
Action Input: df['Age'].mean()29.6991176470588229.69911764705882 is the average age of the passengers.
Final Answer: The average age of the passengers is 29.69911764705882.

> Finished chain.


'The average age of the passengers is 29.69911764705882.'

In [ ]:
agent.run("남성과 여성의 비율은?")



> Entering new AgentExecutor chain...
Thought: I need to find the total number of males and females, and then calculate the ratio between them.
Action: python_repl_ast
Action Input: df['Sex'].value_counts()Sex
male      577
female    314
Name: count, dtype: int64577 males and 314 females, now I need to calculate the ratio.
Action: python_repl_ast
Action Input: df['Sex'].value_counts(normalize=True)Sex
male      0.647587
female    0.352413
Name: proportion, dtype: float640.647587 is the proportion of males and 0.352413 is the proportion of females.
Final Answer: The ratio of males to females is approximately 1.84:1.

> Finished chain.


'The ratio of males to females is approximately 1.84:1.'

In [ ]:
agent.run("객실 등급과 성별에 따른 생존자 수를 계산해줘")



> Entering new AgentExecutor chain...
Thought: I need to group the dataframe by Pclass and Sex, then sum the Survived column
Action: python_repl_ast
Action Input: df.groupby(['Pclass', 'Sex'])['Survived'].sum()Pclass  Sex   
1       female    91
        male      45
2       female    70
        male      17
3       female    72
        male      47
Name: Survived, dtype: int64I now know the final answer
Final Answer: 객실 등급과 성별에 따른 생존자 수는 다음과 같습니다:
|   | Pclass | Sex    |   Survived |
|---:|-------:|:-------|-----------:|
|  0 |      1 | female |         91 |
|  1 |      1 | male   |         45 |
|  2 |      2 | female |         70 |
|  3 |      2 | male   |         17 |
|  4 |      3 | female |         72 |
|  5 |      3 | male   |         47 |

> Finished chain.


'객실 등급과 성별에 따른 생존자 수는 다음과 같습니다:\n|   | Pclass | Sex    |   Survived |\n|---:|-------:|:-------|-----------:|\n|  0 |      1 | female |         91 |\n|  1 |      1 | male   |         45 |\n|  2 |      2 | female |         70 |\n|  3 |      2 | male   |         17 |\n|  4 |      3 | female |         72 |\n|  5 |      3 | male   |         47 |'